<a href="https://colab.research.google.com/github/filipecalegario/stylegan2-ada-experiments/blob/main/StyleGAN2_Closed_Form_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3: Feature Vectors



In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch
%cd stylegan2-ada-pytorch

!pip install ninja

In [ ]:
!wget http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-cat-config-f.pkl

In [ ]:
import os
import io

import numpy as np
import PIL.Image
import cv2
import IPython.display
import dnnlib
import torch

import legacy

In [ ]:
network_pkl = '/content/drive/MyDrive/GDL-studies/ERN-FUFI/modelo/ernesto-inicia-network-snapshot-000016.pkl'

In [ ]:
device = torch.device('cuda')
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)

## Generate Feature Vectors

In [ ]:
modulate = {
    k[0]: k[1]
    for k in G.named_parameters()
    if "affine" in k[0] and "torgb" not in k[0] and "weight" in k[0] or ("torgb" in k[0] and "b4" in k[0] and "weight" in k[0] and "affine" in k[0])
}

weight_mat = []
for k, v in modulate.items():
    weight_mat.append(v)

W = torch.cat(weight_mat, 0)
eigvec = torch.linalg.svd(W).Vh.to("cpu")

torch.save({"ckpt": network_pkl, "eigvec": eigvec}, '/content/drive/MyDrive/GDL-studies/ERN-FUFI/modelo/ern-feature-vectors.pt')

In [ ]:
print(eigvec.shape) # how many dimensions
print(eigvec[0])

## Applying a feature vector to an image

In [ ]:
def imshow(images, col, viz_size=1024):
  """Shows images in one figure."""
  num, height, width, channels = images.shape
  assert num % col == 0
  row = num // col
  # print(num,height,width,channels)

  fused_image = np.zeros((viz_size * row, viz_size * col, channels), dtype=np.uint8)

  for idx, image in enumerate(images):
    i, j = divmod(idx, col)
    y = i * viz_size
    x = j * viz_size
    if height != viz_size or width != viz_size:
      image = cv2.resize(image, (viz_size, viz_size))
    fused_image[y:y + viz_size, x:x + viz_size] = image

  fused_image = np.asarray(fused_image, dtype=np.uint8)
  data = io.BytesIO()
  PIL.Image.fromarray(fused_image).save(data, 'jpeg')
  im_data = data.getvalue()
  disp = IPython.display.display(IPython.display.Image(im_data))
  return disp

In [ ]:
seed = 3923
z = np.random.RandomState(seed).randn(1, G.z_dim)

truncation_psi = 1.0
noise_mode = 'const' # 'const', 'random', 'none'

outdir = '/content/output/'
os.makedirs(outdir, exist_ok=True)

# no labels
label = torch.zeros([1, G.c_dim], device=device)

z = torch.from_numpy(z).to(device)
img_gpu = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
img = (img_gpu.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')

imshow(img.cpu(),col=1)

You’ll see something referred to as `degree` when discussing vectors. It’s probably easier to think of this as strength. Strength can be a negative or positive value, and the large the value (in either direction) the stronger the effect on the vector.

In [ ]:
0.5, 0.5 (-0.01*1000,0.1)

In [ ]:
seed = 3923
z = np.random.RandomState(seed).randn(1, G.z_dim)
z = torch.from_numpy(z)

degree = 10.0 # we'll do positive and negative
vector_index = 10 # any number 0-511

current_eigvec = eigvec[vector_index]
direction = degree * current_eigvec

z0 = z - direction # move z in negative direction
z1 = z # just z
z2 = z + direction # move z in positive direction

zs = torch.cat((z0,z1,z2)).to(device)
# print(zs.shape)

img = G(zs, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png') # uncomment to save images

imshow(img.cpu(),col=3)

In [ ]:
seed = 2991
z = np.random.RandomState(seed).randn(1, G.z_dim)
z = torch.from_numpy(z)

degree = 10.0 # we'll do positive and negative
vector_index = 100

current_eigvec = eigvec[vector_index]
direction = degree * current_eigvec

z = z.cpu()

z0 = z - direction # move z in negative direction
z1 = z - (direction/2) # move z in negative direction (half way)
z2 = z # just z
z3 = z + (direction/2) # move z in positive direction
z4 = z + direction # move z in positive direction

zs = torch.cat((z0,z1,z2,z3,z4)).to(device)

img = G(zs, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
# PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png') # uncomment to save images

imshow(img.cpu(),col=5)



What if we looked at just the poles and interpolated in the w space?

In [ ]:
def lerp(zs, steps):
    out = []
    for i in range(len(zs)-1):
        for index in range(steps):
            t = index/float(steps)
            out.append(zs[i+1]*t + zs[i]*(1-t))
    return out

In [ ]:
zs = [z0,z4]
ws = []

for z_idx, z in enumerate(zs):
    z = z.to(device)
    w = G.mapping(z, label, truncation_psi=truncation_psi, truncation_cutoff=8)
    ws.append(w)

frame_ws = lerp(ws, 5)

ws = torch.cat((frame_ws[0], frame_ws[1], frame_ws[2], frame_ws[3], frame_ws[4]))

img = G.synthesis(ws, noise_mode=noise_mode, force_fp32=True)
img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)

imshow(img.cpu(),col=5)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

For homework/fun, maybe convert thes to videos based on what we’ve done earlier in this class.